In [10]:
import json
import os
import pandas as pd

In [11]:
# Get list of all files in the directory
files = os.listdir('./Filtered_Output/')
jsonl_files = [file for file in files if file.endswith('.jsonl')]
print(jsonl_files)

['dataset_gpt-4_0.8.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.4.jsonl', 'dataset_Salesforce_codegen-2B-mono_1.0.jsonl', 'dataset_bigcode_starcoder_0.8.jsonl', 'dataset_gpt-3.5-turbo_0.8.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.6.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.2.jsonl', 'dataset_Salesforce_codegen-2B-mono_0.0.jsonl', 'dataset_Salesforce_codegen25-7b-mono_1.0.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.4.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.6.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.2.jsonl', 'dataset_Salesforce_codegen25-7b-mono_0.8.jsonl', 'dataset_gpt-4_0.0.jsonl', 'dataset_gpt-3.5-turbo_0.2.jsonl', 'dataset_bigcode_starcoder_0.2.jsonl', 'dataset_gpt-3.5-turbo_0.0.jsonl', 'dataset_gpt-4_0.2.jsonl', 'dataset_gpt-4_0.6.jsonl', 'dataset_gpt-3.5-turbo_1.0.jsonl', 'dataset_bigcode_starcoder_0.6.jsonl', 'dataset_gpt-3.5-turbo_0.4.jsonl', 'dataset_gpt-4_1.0.jsonl', 'dataset_bigcode_starcoder_0.4.jsonl', 'dataset_gpt-3.5-turbo_0.6.jsonl', 'dataset

In [12]:
def get_result(file_path):
    df = pd.read_csv(file_path)
    test_success = None
    test_vulnerability = None
    for index, row in df.iterrows():
        if 'correctness' in row['TestName']:
            test_success = row['Result']
        if 'vulnerability' in row['TestName']:
            test_vulnerability = row['Result']

    return test_success, test_vulnerability

In [13]:
for file_name in jsonl_files:
    # if 'gpt'  not in file_name:
    #     continue    

    model_name = '_'.join(file_name.split('.jsonl')[0].split('_')[1:-1]).replace("Salesforce_", "")
    print(model_name)
    temp = file_name.split('.jsonl')[0].split('_')[-1]
    with open('../Generation/Filtered_Output/' + file_name, 'r') as f:
        data = [json.loads(line) for line in f.readlines()]

    count = 0
    for i in range(len(data)):
        id = data[i]['id'].replace('.py', '')
        ids = id.split('_')
        ids.insert(1, 'test')
        
        id = '_'.join(ids[0:2]+ids[3:])
        technique =  data[i]['technique']
        source = data[i]['source']

        if 'gpt' in model_name:
        
            for j in range(len(data[i]['output']['choices'])):
                result_file = f"./TestModelsResults/{model_name}_{temp}_R{j+1}_{id}_results.csv"
                test_success = None
                test_vulnerability = None
                if os.path.exists(result_file):
                    test_success, test_vulnerability = get_result(result_file)
                
                data[i]['output']['choices'][j]['test_success'] = test_success
                data[i]['output']['choices'][j]['test_vulnerability'] = test_vulnerability
                if not data[i]['output']['choices'][j]['compilable'] and data[i]['output']['choices'][j]['test_success'] == "success":
                    print(result_file)
        else:
            for j in range(len(data[i]['output'])):
                result_file = f"./TestModelsResults/{model_name}_{temp}_R{j+1}_{id}_results.csv"
                test_success = None
                test_vulnerability = None
                if os.path.exists(result_file):
                    test_success, test_vulnerability = get_result(result_file)
                
                data[i]['output'][j]['test_success'] = test_success
                data[i]['output'][j]['test_vulnerability'] = test_vulnerability
                if not data[i]['output'][j]['compilable'] and data[i]['output'][j]['test_success'] == "success":
                    print(result_file)

    with open('./Test_Results/' + file_name, 'w') as f:
        for item in data:
            f.write("%s\n" % json.dumps(item))



            



gpt-4
./TestModelsResults/gpt-4_0.8_R8_Matching_test_A_cwe502_0_results.csv
./TestModelsResults/gpt-4_0.8_R6_Matching_test_SO_cwe327_0_results.csv
./TestModelsResults/gpt-4_0.8_R2_Matching_test_SO_cwe078_0_results.csv
./TestModelsResults/gpt-4_0.8_R3_Assertion_test_A_cwe200_0_results.csv
./TestModelsResults/gpt-4_0.8_R4_Assertion_test_A_cwe200_0_results.csv
./TestModelsResults/gpt-4_0.8_R5_Assertion_test_A_cwe200_0_results.csv
./TestModelsResults/gpt-4_0.8_R8_Assertion_test_A_cwe200_0_results.csv
./TestModelsResults/gpt-4_0.8_R10_Assertion_test_A_cwe200_0_results.csv
./TestModelsResults/gpt-4_0.8_R9_Assertion_test_A_cwe020_0_results.csv
./TestModelsResults/gpt-4_0.8_R6_Assertion_test_A_cwe434_0_results.csv
./TestModelsResults/gpt-4_0.8_R3_Assertion_test_A_cwe730_0_results.csv
./TestModelsResults/gpt-4_0.8_R5_Assertion_test_A_cwe730_0_results.csv
./TestModelsResults/gpt-4_0.8_R6_Assertion_test_A_cwe730_0_results.csv
./TestModelsResults/gpt-4_0.8_R7_Assertion_test_A_cwe730_0_results.csv
